# A Tour of bitformat

A number of classes are available in bitformat to store and manipulate binary data.

* ``Bits`` - An immutable container of binary data.
* ``Dtype`` - A data type that gives an interpretation to binary data.
* ``Array`` - A container for contiguously allocated `Bits` objects with the same `Dtype`.

These are the building blocks for more complex fields that can be used to make a binary format.

* ``Field`` - Either one value or an array, with a single data type, with an optional name and value.
* ``Format`` - A sequence of other FieldTypes, with an optional name.

For this tour we'll first install `bitformat` and import these classes::


In [1]:
%%capture
!pip install git+https://github.com/scott-griffiths/bitformat
from bitformat import Bits, Dtype, Array, Field, Format

Note that we are pip installing directly from the repository, as the library is still in alpha so PyPI probably won't have the latest version.

## Bits

The ``Bits`` class represents an immutable sequence of bits, similar to how the built-in ``bytes`` is an immutable sequence of bytes,
and a ``str`` is an immutable sequence of characters.

There are several builder class methods used to create ``Bits`` objects.

| Method name           | Description                                |
|-----------------------|--------------------------------------------|
| `Bits.pack(dtype, value)` | Combine a data type with a value.        |
| `Bits.from_string(s)` | Use a formatted string.                    |
| `Bits.from_bytes(b)`  | Directly from a `bytes` object.            |
| `Bits.from_iterable(i)` | Converts each element to a single bit.    |
| `Bits.zeros(n)`       | Initialise with zero bits.                 |
| `Bits.ones(n)`        | Initialise with one bits.                  |
| `Bits.join(iterable)` | Concatenate from an iterable such as a list.|


The ``Bits`` constructor can be used as a shortcut for the ``from_string`` method, so ``Bits(s)`` and ``Bits.from_string(s)`` are equivalent.

Creating from a string is often convenient and quite powerful.
The string can be a binary, octal or hexadecimal literal by starting with ``'0b'``, ``'0o'`` or ``'0x'`` respectively.
It can be a string that uses various data types of integer or floating point values, and it can be a sequence of tokens separated by commas. ::

In [2]:
a = Bits('0b110')  # A 3-bit binary string
b = Bits('0xabcde')  # A 20-bit hexadecimal string
c = Bits('bytes=b"abcdefgh"') # An 8 byte bytes object
d = Bits('f32=13.5')  # A 32-bit IEEE floating point number
e = Bits('i7=-31')  # A 7-bit signed integer
f = Bits('0b001, u32=90, 0x5e')  # Three Bits objects concatenated together


Finally a data type can be used to create a `Bits` object by using the `pack` class method. ::

In [3]:
g = Bits.pack('u8', 65)  # An 8-bit unsigned integer with the value 65
h = Bits.pack('hex', 'abcde')  # A 20-bit hexadecimal string
i = Bits.pack('bytes', b'hello')  # A 40-bit binary string
j = Bits.pack('f16', -13.81)  # A 16-bit IEEE floating point number



The first parameter of ``pack`` is the data-type, which can be either a ``Dtype`` or a string that can be used to create one.
The second parameter is a value that makes sense for that data type, which could be a binary string, a floating point number, an integer etc. depending on the ``Dtype``.

Once you've created your ``Bits`` object there is a rich API for manipulating and interpreting the data.
One fundamental thing to do is to interpret the binary data according to a format or data-type; essentially the opposite to how the ``pack`` method works. ::

In [4]:
g.unpack('u8')  # Returns 65

65

In [5]:
h.unpack(['hex20'])  # Returns ['abcde']

['abcde']

The ``unpack`` method is quite powerful and is a bit of a sledgehammer for these simple cases, so as a shortcut you can use properties that are available for simple dtypes.

In [6]:
g.u

65

In [7]:
h.hex

'abcde'

Of course the ``Bits`` object is just a collection of bits and doesn't know how it was created, so any interpretation that makes sense is allowed

In [8]:
a.unpack('oct')  # an octal string

'6'

In [9]:
b.unpack('u')  # an unsigned int

703710

In [10]:
c.unpack('f_le64')  # a 64-bit little-endian IEEE floating point number

8.540883223036124e+194

In [11]:
d.unpack('hex')

'41580000'

In [12]:
e.unpack('bin')

'1100001'

## Constructing a Format

Let's say you have a specification for a binary file type (or maybe a packet specification etc.) and you want to quickly and easily parse and create from the spec in Python. For this example I'm going to use a header from the MPEG-2 video standard. Here's how the header is described in the standard:

|sequence_header() | No. of bits | Mnemonic| 
|-----------------------------------|--------------|----------|
|sequence_header_code | 32 | bslbf |
|horizontal_size_value | 12 | uimsbf | 
|vertical_size_value | 12 | uimsbf |
|aspect_ratio_information | 4 | uimsbf | 
|frame_rate_code | 4 | uimsbf |
|bit_rate_value | 18 | uimsbf | 
|marker_bit | 1 | bslbf |
|vbv_buffer_size_value | 10 | uimsbf |
|constrained_parameters_flag | 1 | bslbf |
|load_intra_quantiser_matrix | 1 | uimsbf |

The mnemonics mean things like uimsbf = 'Unsigned integer, most significant bit first'.

Converting this to a `Format` is simple:


In [13]:
f_str = """
sequence_header: [
    sequence_header_code: const hex32 = 0x000001b3,
    horizontal_size_value: u12,
    vertical_size_value: u12,
    aspect_ratio_information: u4,
    frame_rate_code: u4,
    bit_rate_value: u18,
    marker_bit: bool,
    vbv_buffer_size_value: u10,
    constrained_parameters_flag: bool,
    load_intra_quantiser_matrix: u1
]
"""
f = Format(f_str)          

Here we created the `Format` from a single string, containing comma separated fields. This is often convenient, and allows easy storage of formats, but parameterised creation methods are also available for both the fields and the format.

We have set the first field to be a `const`, meaning that its value should not be changed. For the other fields we can give them values manually with the `pack` method:

In [14]:
f.pack([352, 288, 0, 1, 104000, True, 880, False, 1])
print(f)

sequence_header: [
    sequence_header_code: const hex32 = 000001b3,
    horizontal_size_value: u12 = 352,
    vertical_size_value: u12 = 288,
    aspect_ratio_information: u4 = 0,
    frame_rate_code: u4 = 1,
    bit_rate_value: u18 = 104000,
    marker_bit: bool = True,
    vbv_buffer_size_value: u10 = 880,
    constrained_parameters_flag: bool = False,
    load_intra_quantiser_matrix: u1 = 1
]


We can now query and modify the values directly, and output it as a `Bits`, or convert to `bytes`

In [15]:
f['bit_rate_value'].value /= 4
b = f.to_bytes()
print(b)

b'\x00\x00\x01\xb3\x16\x01 \x01\x19d;\x82'


The `Format` works symmetrically, so can be used to parse as well as create binary data.
So if we first clear the data (which removes everything except the const field) we can now parse the bytes we just created and check that the change we made to the `bit_rate_value` field was indeed correctly encoded.

In [16]:
f.clear()
f.parse(b)
print(f)

sequence_header: [
    sequence_header_code: const hex32 = 000001b3,
    horizontal_size_value: u12 = 352,
    vertical_size_value: u12 = 288,
    aspect_ratio_information: u4 = 0,
    frame_rate_code: u4 = 1,
    bit_rate_value: u18 = 26000,
    marker_bit: bool = True,
    vbv_buffer_size_value: u10 = 880,
    constrained_parameters_flag: bool = False,
    load_intra_quantiser_matrix: u1 = 1
]


# Worked Examples
Below are a few examples of using the bitformat module, as I always find that a good example can help more than a lengthy reference manual.

## Hamming distance
The Hamming distance between two bitstrings is the number of bit positions in which the two bitstrings differ. So for example the distance between 0b00110 and 0b01100 is 2 as the second and fourth bits are different.

Let's write a function that calculates the Hamming weight of two bitstrings.

In [17]:
def hamming_weight(a, b):
    return (Bits(a)^b).count(True)

hamming_weight('0b00110', '0b01100')

2

Er, that's it. The `^` is a bit-wise exclusive or, which means that the bits in `a^b` are only set if they differ in `a` and `b`. The `count` method just counts the number of 1 (or True) bits.

## Sieve of Eratosthenes

The sieve of Eratosthenes is an ancient (and very inefficient) method of finding prime numbers. The algorithm starts with the number 2 (which is prime) and marks all of its multiples as not prime, it then continues with the next unmarked integer (which will also be prime) and marks all of its multiples as not prime.


So to find all primes under a hundred million you could write:

In [21]:
import math

# Create a Bits with a hundred million 'one' bits
limit = 100_000_000
is_prime = Bits.ones(limit)
# Manually set 0 and 1 to be not prime.
is_prime = is_prime.set(False, [0, 1])
# For every other integer, if it's set as prime then unset all of its multiples
for i in range(2, math.ceil(math.sqrt(limit))):
    if is_prime[i]:
        is_prime = is_prime.set(False, range(i*i, limit, i))

print(f"There are {is_prime.count(True)} primes less than {limit},")
print(f"the largest one of which is {is_prime.rfind('0b1')}")
print(f"and there are {len(list(is_prime.find_all('0b101')))} twin primes.")

There are 5761455 primes less than 100000000,
the largest one of which is 99999989
and there are 440312 twin primes.


We find the largest prime with a reverse find (`rfind`) looking for a single set bit. For twin primes (primes which differ by 2) we use `find_all` to look for the binary sequence 101 which returns a generator for the bit positions.

To see the pattern of the primes we could use the pretty print method:

In [29]:
is_prime[0:1000].pp()

I'll leave optimising the algorithm as an exercise for the reader, but it illustrates both bit checking and setting. One reason you might want to use bitformat for this purpose (instead of a plain list for example) is that the million bits only take up a million bits in memory, whereas for a list of integers it would be much more.